In [142]:
import numpy as np
import pandas as pd
import matplotlib
import requests
from pandas.tseries.offsets import BDay
import yfinance as yf
from datetime import datetime
import pickle

# Loading the dataset:


In [26]:
df_initial= pd.read_csv("/Users/manonlaffly/code/cobergmann/MA_PREDICTOR/raw_data/210906_LeWagon_finalproject_v2.csv")

In [27]:
df_initial = df_initial.rename(columns={'Acquiror Primary Ticker Symbol': 'Acquiror_Primary_Ticker_Symbol', 'Date Announced': 'Date_Announced'})

In [28]:
df_initial = df_initial.dropna(subset=['Acquiror_Primary_Ticker_Symbol'])

In [29]:
df_initial['Date_Announced'] = pd.to_datetime(df_initial['Date_Announced'])

Here is the initial dataframe with date as datetime and certain names of columns modified:

In [30]:
df_initial


,SDC Deal No,Date_Announced,"Deal Value\n(USD, Millions)",Acquiror Full Name,Acquiror_Primary_Ticker_Symbol,Target Full Name,Target Nation,Acquiror Nation,Target Public Status,Acquiror TRBC Industry,Target TRBC Industry,Percentage of Shares Held at Announcement,Percentage of Shares Acquired in Transaction,Consideration Offered\n('|'),Deal Attitude,Acquisition Techniques\n('|'),Acquiror Financial Advisors Name\n('|'),Target Financial Advisors Name\n('|')
0,1631732020,2005-01-01,12.00,Plains All American Pipeline LP,PAA,Shell Pipeline Co LP-Crude Oil Pipeline Assets...,United States,United States,Subsidiary,Oil & Gas Refining and Marketing,Oil & Gas Transportation Services,NaN,100.0,Cash|Cash Only,Friendly,Financial Acquiror|Divestiture,NaN,NaN
1,1653652020,2005-01-01,NaN,Regis Corp,RGS,Scot Lewis Schools,United States,United States,Private,Personal Services,Professional & Business Education,NaN,100.0,Unspecified,Friendly,Not Applicable,NaN,NaN
2,1670089020,2005-01-01,"1,700.00",Prudential Financial Inc,PRU,Prudential Financial Inc,United States,United States,Public,Life & Health Insurance,Life & Health Insurance,NaN,0.0,Cash|Cash Only,No Applicable,Open Market Purchase|Repurchase,NaN,NaN
3,1949421020,2005-01-01,NaN,Google Inc,GOOG,PhatBits,United States,United States,Private,Online Services,Software,NaN,100.0,Unspecified,Friendly,Not Applicable,NaN,NaN
4,1652453040,2005-01-01,20.03,Graco Inc,GGG,Gusmer Europe SL,Spain,United States,Private,Industrial Machinery & Equipment,Commodity Chemicals,NaN,100.0,Cash|Cash Only,Friendly,Not Applicable,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29314,3794100020,2021-01-09,NaN,Generac Holdings Inc,GNRC,Apricity Code Corp,United States,United States,Private,Electrical Components & Equipment,Construction & Engineering,NaN,100.0,Unspecified,Friendly,Financial Acquiror,NaN,NaN
29315,3793471020,2021-02-09,179.50,Builders FirstSource Inc,BLDR,California Trusframe LLC,United States,United States,Private,Home Improvement Products & Services Retailers,Construction & Engineering,NaN,100.0,Cash|Cash Only,Friendly,Financial Acquiror,NaN,NaN
29316,3793513020,2021-02-09,NaN,Pennant Group Inc,PNTG,Open Heart Hospice,United States,United States,Private,Healthcare Facilities & Services,Healthcare Facilities & Services,NaN,100.0,Unspecified,Friendly,Not Applicable,NaN,NaN
29317,3793867040,2021-03-09,NaN,Herc Holdings Inc,HRI,Dwight Crane Ltd,Canada,United States,Private,"Passenger Transportation, Ground & Sea",Business Support Services,NaN,100.0,Unspecified,Friendly,Financial Acquiror,NaN,NaN


In [100]:
msft = yf.Ticker("AAPL")

In [101]:
hist = msft.history(start="2020-01-01", end="2020-04-30")[['Open', 'Close']]

In [102]:
hist

,Open,Close
Date,,
2020-01-02,73.082501,74.096443
2020-01-03,73.307008,73.376083
2020-01-06,72.478106,73.960770
2020-01-07,73.970641,73.612930
2020-01-08,73.309478,74.797081
...,...,...
2020-04-23,68.218751,68.011032
2020-04-24,68.547636,69.974472
2020-04-27,69.685153,70.023941


# Extracting the right interval of time

Then we do a copy of the dataframe by selecting the ticker and the announcement date for each merger:

In [31]:
df_min_max = df_initial[['Acquiror_Primary_Ticker_Symbol', 'Date_Announced']].copy()

We add another column that we will call date_max, which for now is just a copy of the date of announcement. 

In [32]:
df_min_max['date_max']= df_initial['Date_Announced']

We create a new dataframe that is the result of a groupby function and where the column 'Date_Announced' represent the the most ancient announcement day for a merger and date_max is the most recent announcement day for a merger. Those 2 date are expressed for each ticker:

In [66]:
df_restricted = df_min_max.groupby('Acquiror_Primary_Ticker_Symbol').agg({
                "Date_Announced": min,
                "date_max": max
            })

In [67]:
df_restricted

,Date_Announced,date_max
Acquiror_Primary_Ticker_Symbol,,
4704,2005-06-14,2008-02-25
950140,2017-01-02,2017-01-02
A,2005-01-20,2021-02-03
AAAC,2007-07-25,2007-07-25
AABA,2018-07-06,2018-07-06
...,...,...
ZTS,2014-11-17,2020-08-04
ZVO,2019-04-04,2019-04-04
ZY,2021-05-28,2021-05-28


Then, we create two new columns that will represent 300 days before the first merger and 50 days after the last merger for each company : 

In [68]:
df_restricted['30_days_before']= df_restricted['Date_Announced'] - BDay(30)

In [69]:
df_restricted['30_days_after']= df_restricted['date_max'] + BDay(30)

Here is the final restricted dataframe with the correct interval of time:

In [70]:
df_restricted

,Date_Announced,date_max,30_days_before,30_days_after
Acquiror_Primary_Ticker_Symbol,,,,
4704,2005-06-14,2008-02-25,2005-05-03,2008-04-07
950140,2017-01-02,2017-01-02,2016-11-21,2017-02-13
A,2005-01-20,2021-02-03,2004-12-09,2021-03-17
AAAC,2007-07-25,2007-07-25,2007-06-13,2007-09-05
AABA,2018-07-06,2018-07-06,2018-05-25,2018-08-17
...,...,...,...,...
ZTS,2014-11-17,2020-08-04,2014-10-06,2020-09-15
ZVO,2019-04-04,2019-04-04,2019-02-21,2019-05-16
ZY,2021-05-28,2021-05-28,2021-04-16,2021-07-09


# Extracting stock prices

Now, we extract the list of unique tickers from the df_initial :

In [38]:
unique_ac_code= df_initial['Acquiror_Primary_Ticker_Symbol'].unique().tolist()

In [39]:
#Here is the number of unique companies. 
len(unique_ac_code)

5673

Verify if duplicates in both list. 

In [27]:
def checkIfDuplicates_2(listOfElems):
    ''' Check if given list contains any duplicates '''    
    setOfElems = set()
    for elem in listOfElems:
        if elem in setOfElems:
            return True
        else:
            setOfElems.add(elem)         
    return False

In [28]:
result = checkIfDuplicates_2(unique_ac_code)
if result:
    print('Yes, list contains duplicates')
else:
    print('No duplicates found in list')   

No duplicates found in list


In [ ]:
def fix_datetime_on_dataframe(df):
    df['date'] = pd.to_datetime(df['date'])
    df.set_index(keys='date', inplace=True)


In [ ]:
def create_stock_df_of_company(company_code):
    """This function will retrieve a dataframe with the date as index and
    with the closing and op """
    url = f'http://iex.lewagon.com/stable/stock/{company_code}/chart/max'
    df = pd.read_json(url)
    fix_datetime_on_dataframe(df)
    df['company_code'] = company_code
    return df[['close', 'open']]

In [122]:
list_df=[]

In [123]:
list_errors=[]

In [124]:
for code in unique_ac_code:
        ticker = yf.Ticker(code)
        date_min=df_restricted.loc[code, '30_days_before'].strftime("%Y-%m-%d")
        date_max=df_restricted.loc[code, '30_days_after'].strftime("%Y-%m-%d")
        stock_data = ticker.history(start=date_min, end=date_max)[['Open', 'Close']]
        if len(stock_data) != 0:
            list_df.append(stock_data)
        else:
            list_errors.append(str(code))

- NPBC: No data found for this date range, symbol may be delisted
- MHA: No data found for this date range, symbol may be delisted
- PKY: No data found for this date range, symbol may be delisted
- GBTD: No data found, symbol may be delisted
- NAVR: No data found for this date range, symbol may be delisted
- SEGB: No data found for this date range, symbol may be delisted
- INMD: Data doesn't exist for startDate = 1124665200, endDate = 1298592000
- MBND: Data doesn't exist for startDate = 1121641200, endDate = 1324339200
- NWS: Data doesn't exist for startDate = 1105401600, endDate = 1356998400
- TUC: No data found for this date range, symbol may be delisted
- LBY: No data found, symbol may be delisted
- XTO: No data found for this date range, symbol may be delisted
- AGL: Data doesn't exist for startDate = 1102291200, endDate = 1137024000
- EK: No data found for this date range, symbol may be delisted
- TESV: No data found, symbol may be delisted
- PPDI: No data found for this date ran

- OCR: No data found for this date range, symbol may be delisted
- ICO: Data doesn't exist for startDate = 1101168000, endDate = 1215471600
- ALY: No data found for this date range, symbol may be delisted
- CNXT: Data doesn't exist for startDate = 1108944000, endDate = 1227225600
- RMLX: No data found for this date range, symbol may be delisted
- VRDM: No data found for this date range, symbol may be delisted
- NTRZ: No data found for this date range, symbol may be delisted
- FBOR: No data found for this date range, symbol may be delisted
- LFGRQ: No data found, symbol may be delisted
- LLL: No data found, symbol may be delisted
- DPTR: No data found for this date range, symbol may be delisted
- GTIV: No data found for this date range, symbol may be delisted
- CKP: No data found for this date range, symbol may be delisted
- PTNW: No data found for this date range, symbol may be delisted
- KMP: No data found for this date range, symbol may be delisted
- CMDJ: No data found, symbol may b

- SBIB: No data found for this date range, symbol may be delisted
- PPP: No data found for this date range, symbol may be delisted
- BDFC: No data found, symbol may be delisted
- ATMI: No data found for this date range, symbol may be delisted
- HTRN: No data found for this date range, symbol may be delisted
- CLCT: No data found, symbol may be delisted
- PCLN: No data found for this date range, symbol may be delisted
- XXIA: No data found for this date range, symbol may be delisted
- CHB: Data doesn't exist for startDate = 1118098800, endDate = 1201737600
- WAVE: Data doesn't exist for startDate = 1118098800, endDate = 1199923200
- XCHCD: No data found, symbol may be delisted
- PER: No data found, symbol may be delisted
- FPO: No data found for this date range, symbol may be delisted
- GEEK: Data doesn't exist for startDate = 1118271600, endDate = 1180652400
- SCBT: No data found for this date range, symbol may be delisted
- CKXE: No data found for this date range, symbol may be delist

- EMGL: No data found for this date range, symbol may be delisted
- TVIN: No data found for this date range, symbol may be delisted
- HSTT: No data found, symbol may be delisted
- HAF: No data found for this date range, symbol may be delisted
- DEVC: No data found for this date range, symbol may be delisted
- AGN: No data found, symbol may be delisted
- CBHI: No data found for this date range, symbol may be delisted
- HLTH: No data found, symbol may be delisted
- IHS: No data found for this date range, symbol may be delisted
- FLIR: No data found, symbol may be delisted
- SNWL: No data found for this date range, symbol may be delisted
- MNCN: No data found, symbol may be delisted
- DNR: No data found, symbol may be delisted
- VITS: No data found for this date range, symbol may be delisted
- SNMD: No data found, symbol may be delisted
- AMDD: No data found, symbol may be delisted
- AFCO: No data found for this date range, symbol may be delisted
- MFCD: No data found for this date range,

- RSTI: No data found for this date range, symbol may be delisted
- CMHT: No data found, symbol may be delisted
- MAEN: No data found for this date range, symbol may be delisted
- ALJ: No data found for this date range, symbol may be delisted
- AFGSV: No data found, symbol may be delisted
- OTRS: No data found, symbol may be delisted
- EPIX: Data doesn't exist for startDate = 1137974400, endDate = 1144969200
- HBDT: No data found for this date range, symbol may be delisted
- JEC: No data found, symbol may be delisted
- PCYC: No data found for this date range, symbol may be delisted
- CFHZ: No data found, symbol may be delisted
- ANX: No data found for this date range, symbol may be delisted
- OEGY: No data found, symbol may be delisted
- PWTC: No data found for this date range, symbol may be delisted
- DOTX: No data found for this date range, symbol may be delisted
- EESV: No data found for this date range, symbol may be delisted
- BBG: No data found for this date range, symbol may be 

...

- DEACD: No data found for this date range, symbol may be delisted
- KFN: No data found for this date range, symbol may be delisted
- CNBC: No data found for this date range, symbol may be delisted
- DWRE: No data found for this date range, symbol may be delisted
- MCIG: No data found, symbol may be delisted
- ANPZ: No data found for this date range, symbol may be delisted
- DUSR: No data found, symbol may be delisted
- DANE: No data found for this date range, symbol may be delisted
- WEFT: No data found, symbol may be delisted
- TFER: No data found for this date range, symbol may be delisted
- FNRG: No data found for this date range, symbol may be delisted
- ARX: No data found for this date range, symbol may be delisted
- PDO: Data doesn't exist for startDate = 1398034800, endDate = 1405292400
- AHP: No data found for this date range, symbol may be delisted
- MSTX: No data found for this date range, symbol may be delisted
- IMMY: No data found for this date range, symbol may be delist

- AAC: Data doesn't exist for startDate = 1414972800, endDate = 1439506800
- POCO: No data found for this date range, symbol may be delisted
- NEMD: Data doesn't exist for startDate = 1415750400, endDate = 1423008000
- NBCB: No data found for this date range, symbol may be delisted
- AHC: No data found, symbol may be delisted
- CAGC: No data found for this date range, symbol may be delisted
- EAGLU: No data found, symbol may be delisted
- HWKR: No data found for this date range, symbol may be delisted
- CLRX: No data found for this date range, symbol may be delisted
- BRDR: No data found, symbol may be delisted
- CNCK: No data found for this date range, symbol may be delisted
- NEPG: No data found, symbol may be delisted
- PN: No data found for this date range, symbol may be delisted
- PDRG: No data found, symbol may be delisted
- GRUB: Data doesn't exist for startDate = 1424304000, endDate = 1541462400
- SYYC: Data doesn't exist for startDate = 1420502400, endDate = 1427756400
- LNTP:

- BIOD: No data found for this date range, symbol may be delisted
- BXLT: No data found for this date range, symbol may be delisted
- CRMV: No data found, symbol may be delisted
- CTRV: No data found, symbol may be delisted
- CHITD: No data found, symbol may be delisted
- SSOL: No data found, symbol may be delisted
- TDCP: No data found for this date range, symbol may be delisted
- CZMR: No data found for this date range, symbol may be delisted
- MEET: No data found, symbol may be delisted
- VLOVD: No data found, symbol may be delisted
- GRSHU: No data found, symbol may be delisted
- MSG: No data found, symbol may be delisted
- HDRAU: No data found, symbol may be delisted
- DFBG: No data found for this date range, symbol may be delisted
- SHRK: Data doesn't exist for startDate = 1465340400, endDate = 1472598000
- SRAQ: No data found, symbol may be delisted
- PRFC: No data found, symbol may be delisted
- FH: No data found for this date range, symbol may be delisted
- SUHE: No data found

- NDVN: Data doesn't exist for startDate = 1515024000, endDate = 1522278000
- NWIN: No data found, symbol may be delisted
- INOLD: No data found, symbol may be delisted
- ACHI: No data found for this date range, symbol may be delisted
- MACQ: Data doesn't exist for startDate = 1516060800, endDate = 1523314800
- PKKW: Data doesn't exist for startDate = 1511308800, endDate = 1518566400
- LITEV: No data found, symbol may be delisted
- TPGE U: No data found, symbol may be delisted
- TEGP: No data found for this date range, symbol may be delisted
- SPEX: No data found, symbol may be delisted
- PSDVV: No data found, symbol may be delisted
- ATACU: No data found, symbol may be delisted
- PCFP: No data found for this date range, symbol may be delisted
- PSDO: No data found, symbol may be delisted
- BOFI: No data found for this date range, symbol may be delisted
- VVUS: No data found, symbol may be delisted
- FUSB.O: No data found, symbol may be delisted
- BPET: No data found, symbol may be del

- CCXX: No data found, symbol may be delisted
- SPAQ: Data doesn't exist for startDate = 1590966000, endDate = 1598223600
- HYAC: Data doesn't exist for startDate = 1590966000, endDate = 1598223600
- FVAC U: No data found, symbol may be delisted
- TZAC: No data found, symbol may be delisted
- DMYT: No data found, symbol may be delisted
- HCCO: No data found, symbol may be delisted
- PTAC: No data found, symbol may be delisted
- FTAC: No data found, symbol may be delisted
- DPHC: No data found, symbol may be delisted
- CFFA: No data found, symbol may be delisted
- ASIN: No data found, symbol may be delisted
- SAQN: No data found, symbol may be delisted
- HPTY: No data found, symbol may be delisted
- GMHI: No data found, symbol may be delisted
- MWK: No data found, symbol may be delisted
- TRNE: No data found, symbol may be delisted
- FEAC: No data found, symbol may be delisted
- KCAC U: No data found, symbol may be delisted
- XSNXD: No data found, symbol may be delisted
- CCA: No data f

In [128]:
(len(unique_ac_code) - len(list_errors)) / len(unique_ac_code)

0.46483342147012163

We take the tickers that don't work with the API out of the list of tickers.

In [129]:
for code in list_errors:
    unique_ac_code.remove(code)

In [130]:
len(unique_ac_code)

2637

Finally, we zip our two lists into a dictionary where each key will be a unique ticker that does work with the API. The value of each key will be a dataframe of the stock prices of this company 300 days before the first merger and 50 days after the last one. 

In [131]:
dictionary = dict(zip(unique_ac_code, list_df))

In [140]:
dictionary["PAA"]["Open"]

Date
2004-11-22    5.939119
2004-11-23    5.970242
2004-11-24    6.060327
2004-11-26    6.142221
2004-11-29    6.101277
                ...   
2020-05-07    7.178367
2020-05-08    7.258922
2020-05-11    7.437934
2020-05-12    7.428983
2020-05-13    7.437934
Name: Open, Length: 3895, dtype: float64

In [143]:
with open('stockdata.pickle', 'wb') as handle:
    pickle.dump(dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [1]:
!pwd

/Users/manonlaffly/code/cobergmann/MA_PREDICTOR/notebooks


In [145]:
pd.read_pickle('stockdata.pickle')

{'PAA':                 Open     Close
 Date                          
 2004-11-22  5.939119  5.958775
 2004-11-23  5.970242  6.047225
 2004-11-24  6.060327  6.183172
 2004-11-26  6.142221  6.119290
 2004-11-29  6.101277  6.129122
 ...              ...       ...
 2020-05-07  7.178367  7.079910
 2020-05-08  7.258922  7.500588
 2020-05-11  7.437934  7.375279
 2020-05-12  7.428983  7.473736
 2020-05-13  7.437934  7.285774
 
 [3895 rows x 2 columns],
 'RGS':                  Open      Close
 Date                            
 2004-11-22  41.451579  41.737072
 2004-11-23  41.856792  41.783115
 2004-11-24  41.672607  41.967308
 2004-11-26  41.948876  41.838364
 2004-11-29  41.810736  41.764687
 ...               ...        ...
 2020-02-04  17.709999  14.990000
 2020-02-05  15.140000  14.820000
 2020-02-06  14.980000  14.860000
 2020-02-07  14.740000  14.150000
 2020-02-10  14.090000  13.390000
 
 [3830 rows x 2 columns],
 'PRU':                  Open      Close
 Date                          